In [1]:
import pandas as pd
import numpy as np

# Wczytanie plików `csv` do ramek danych `pandas`

In [3]:
flights = pd.read_csv("http://www.gagolewski.com/resources/data/nycflights13_flights.csv.gz", comment="#", compression="gzip")

In [4]:
airports = pd.read_csv("http://www.gagolewski.com/resources/data/nycflights13_airports.csv.gz", comment="#", compression="gzip")

In [5]:
planes = pd.read_csv("http://www.gagolewski.com/resources/data/nycflights13_planes.csv.gz", comment="#", compression="gzip")

In [6]:
airlines = pd.read_csv("http://www.gagolewski.com/resources/data/nycflights13_airlines.csv.gz", comment="#", compression="gzip")

In [7]:
weather = pd.read_csv("http://www.gagolewski.com/resources/data/nycflights13_weather.csv.gz", comment="#", compression="gzip")

# Utworzenie ramek danych PySpark

In [1]:
from pyspark import *
from pyspark.sql import *

In [6]:
sc = SparkContext.getOrCreate()
#ss = SQLContext(sc)
#dic = planes.to_dict(orient="list")
#values = list(dic.values())
#rows_list = list(zip(*values))
#row_desc = Row(*(dic.keys()))
#rows = [row_desc(*r) for r in rows_list]
#sc.parallelize(rows).toDF()
#sp = SparkSession.Builder().enableHiveSupport().getOrCreate()

Niestety, nie udało mi się utworzyć ani contextu SQL and SparkSession.

# Utworzenie bazy `SQLite`

In [11]:
import sqlite3

In [12]:
nycflights13_db = sqlite3.connect("nycflights13.db")

In [17]:
pandas_df = {"flights": flights, "airports": airports, "planes":planes, "airlines" : airlines, "weather" : weather}

In [19]:
try: # rzuca wyjatek, gdy juz tabela istnieje (bo wtedy juz nic nie trzeba tworzyc)
    for name, df in pandas_df.items():
        df.to_sql(name, nycflights13_db)
except:
    pass

# Zapytania

## Zapytanie nr 1

### `SQL`

In [33]:
pd.read_sql_query("SELECT DISTINCT engine FROM planes", nycflights13_db)

,engine
0,Turbo-fan
1,Turbo-jet
2,Reciprocating
3,4 Cycle
4,Turbo-shaft
5,Turbo-prop


### `Pandas`

In [34]:
pd.DataFrame({"engine" : planes.engine.unique()})

,engine
0,Turbo-fan
1,Turbo-jet
2,Reciprocating
3,4 Cycle
4,Turbo-shaft
5,Turbo-prop


### `PySpark`

## Zapytanie nr 2

### `SQL`

In [32]:
pd.read_sql_query("SELECT DISTINCT type, manufacturer FROM planes", nycflights13_db)

,type,manufacturer
0,Fixed wing multi engine,EMBRAER
1,Fixed wing multi engine,AIRBUS INDUSTRIE
2,Fixed wing multi engine,BOEING
3,Fixed wing multi engine,AIRBUS
4,Fixed wing multi engine,BOMBARDIER INC
5,Fixed wing single engine,CESSNA
6,Fixed wing multi engine,CESSNA
7,Fixed wing single engine,JOHN G HESS
8,Fixed wing multi engine,GULFSTREAM AEROSPACE
9,Rotorcraft,SIKORSKY


### `Pandas`

In [43]:
pd.DataFrame([(key[0], key[1]) for key, group in planes.groupby(["type", "manufacturer"])])

,0,1
0,Fixed wing multi engine,AIRBUS
1,Fixed wing multi engine,AIRBUS INDUSTRIE
2,Fixed wing multi engine,AVIONS MARCEL DASSAULT
3,Fixed wing multi engine,BEECH
4,Fixed wing multi engine,BOEING
5,Fixed wing multi engine,BOMBARDIER INC
6,Fixed wing multi engine,CANADAIR
7,Fixed wing multi engine,CANADAIR LTD
8,Fixed wing multi engine,CESSNA
9,Fixed wing multi engine,DOUGLAS


### `PySpark`

## Zapytanie nr 3

### `SQL`

In [44]:
pd.read_sql_query("SELECT COUNT(*), engine FROM planes GROUP BY engine", nycflights13_db)

,COUNT(*),engine
0,2,4 Cycle
1,28,Reciprocating
2,2750,Turbo-fan
3,535,Turbo-jet
4,2,Turbo-prop
5,5,Turbo-shaft


### `Pandas`

In [78]:
pd.DataFrame(planes.groupby(["engine"]) \
             .size() \
             .reset_index() \
             .rename(index=str, columns={0:"count"}))

,engine,count
0,4 Cycle,2
1,Reciprocating,28
2,Turbo-fan,2750
3,Turbo-jet,535
4,Turbo-prop,2
5,Turbo-shaft,5


### `PySpark`

## Zapytanie nr 4

### `SQL`

In [57]:
pd.read_sql_query("SELECT COUNT(*), engine, type FROM planes GROUP BY engine, type", nycflights13_db)

,COUNT(*),engine,type
0,2,4 Cycle,Fixed wing single engine
1,5,Reciprocating,Fixed wing multi engine
2,23,Reciprocating,Fixed wing single engine
3,2750,Turbo-fan,Fixed wing multi engine
4,535,Turbo-jet,Fixed wing multi engine
5,2,Turbo-prop,Fixed wing multi engine
6,5,Turbo-shaft,Rotorcraft


### `Pandas`

In [76]:
pd.DataFrame(planes.groupby(["engine", "type"]) \
             .size() \
             .reset_index() \
             .rename(index=str, columns={0: "count"}))

,engine,type,count
0,4 Cycle,Fixed wing single engine,2
1,Reciprocating,Fixed wing multi engine,5
2,Reciprocating,Fixed wing single engine,23
3,Turbo-fan,Fixed wing multi engine,2750
4,Turbo-jet,Fixed wing multi engine,535
5,Turbo-prop,Fixed wing multi engine,2
6,Turbo-shaft,Rotorcraft,5


### `PySpark`

## Zapytanie nr 5

### `SQL`

In [80]:
pd.read_sql_query("SELECT MIN(year), AVG(year), MAX(year), engine, manufacturer FROM planes GROUP BY engine, manufacturer", nycflights13_db)

,MIN(year),AVG(year),MAX(year),engine,manufacturer
0,1975.0,1975.000000,1975.0,4 Cycle,CESSNA
1,NaN,NaN,NaN,4 Cycle,JOHN G HESS
2,NaN,NaN,NaN,Reciprocating,AMERICAN AIRCRAFT INC
3,2007.0,2007.000000,2007.0,Reciprocating,AVIAT AIRCRAFT INC
4,NaN,NaN,NaN,Reciprocating,BARKER JACK L
5,1959.0,1971.142857,1983.0,Reciprocating,CESSNA
6,2007.0,2007.000000,2007.0,Reciprocating,CIRRUS DESIGN CORP
7,1959.0,1959.000000,1959.0,Reciprocating,DEHAVILLAND
8,1956.0,1956.000000,1956.0,Reciprocating,DOUGLAS
9,2007.0,2007.000000,2007.0,Reciprocating,FRIEDEMANN JON


### `Pandas`

In [86]:
pd.DataFrame([(key[0], key[1], group.year.min(), group.year.mean(), group.year.max()) \
              for key, group in planes.groupby(["engine", "manufacturer"])]) \
    .rename(columns={0: "engine", 1: "manufacturer", 2 : "min(year)", 3: "avg(year)", 4: "max(year)"})

,engine,manufacturer,min(year),avg(year),max(year)
0,4 Cycle,CESSNA,1975.0,1975.000000,1975.0
1,4 Cycle,JOHN G HESS,NaN,NaN,NaN
2,Reciprocating,AMERICAN AIRCRAFT INC,NaN,NaN,NaN
3,Reciprocating,AVIAT AIRCRAFT INC,2007.0,2007.000000,2007.0
4,Reciprocating,BARKER JACK L,NaN,NaN,NaN
5,Reciprocating,CESSNA,1959.0,1971.142857,1983.0
6,Reciprocating,CIRRUS DESIGN CORP,2007.0,2007.000000,2007.0
7,Reciprocating,DEHAVILLAND,1959.0,1959.000000,1959.0
8,Reciprocating,DOUGLAS,1956.0,1956.000000,1956.0
9,Reciprocating,FRIEDEMANN JON,2007.0,2007.000000,2007.0


### PySpark

## Zapytanie nr 6

### `SQL`

In [89]:
pd.read_sql_query("SELECT * FROM planes WHERE speed IS NOT NULL", nycflights13_db)

,index,tailnum,year,type,manufacturer,model,engines,seats,speed,engine
0,424,N201AA,1959.0,Fixed wing single engine,CESSNA,150,1,2,90.0,Reciprocating
1,427,N202AA,1980.0,Fixed wing multi engine,CESSNA,421C,2,8,90.0,Reciprocating
2,821,N350AA,1980.0,Fixed wing multi engine,PIPER,PA-31-350,2,8,162.0,Reciprocating
3,893,N364AA,1973.0,Fixed wing multi engine,CESSNA,310Q,2,6,167.0,Reciprocating
4,1027,N378AA,1963.0,Fixed wing single engine,CESSNA,172E,1,4,105.0,Reciprocating
5,1037,N381AA,1956.0,Fixed wing multi engine,DOUGLAS,DC-7BF,4,102,232.0,Reciprocating
6,1190,N425AA,1968.0,Fixed wing single engine,PIPER,PA-28-180,1,4,107.0,Reciprocating
7,1430,N508AA,1975.0,Rotorcraft,BELL,206B,1,5,112.0,Turbo-shaft
8,1480,N519MQ,1983.0,Fixed wing single engine,CESSNA,A185F,1,6,127.0,Reciprocating
9,1515,N525AA,1980.0,Fixed wing multi engine,PIPER,PA-31-350,2,8,162.0,Reciprocating


### `Pandas`

In [186]:
planes.loc[~planes.speed.isnull(), :].reset_index()

,index,tailnum,year,type,manufacturer,model,engines,seats,speed,engine
0,424,N201AA,1959.0,Fixed wing single engine,CESSNA,150,1,2,90.0,Reciprocating
1,427,N202AA,1980.0,Fixed wing multi engine,CESSNA,421C,2,8,90.0,Reciprocating
2,821,N350AA,1980.0,Fixed wing multi engine,PIPER,PA-31-350,2,8,162.0,Reciprocating
3,893,N364AA,1973.0,Fixed wing multi engine,CESSNA,310Q,2,6,167.0,Reciprocating
4,1027,N378AA,1963.0,Fixed wing single engine,CESSNA,172E,1,4,105.0,Reciprocating
5,1037,N381AA,1956.0,Fixed wing multi engine,DOUGLAS,DC-7BF,4,102,232.0,Reciprocating
6,1190,N425AA,1968.0,Fixed wing single engine,PIPER,PA-28-180,1,4,107.0,Reciprocating
7,1430,N508AA,1975.0,Rotorcraft,BELL,206B,1,5,112.0,Turbo-shaft
8,1480,N519MQ,1983.0,Fixed wing single engine,CESSNA,A185F,1,6,127.0,Reciprocating
9,1515,N525AA,1980.0,Fixed wing multi engine,PIPER,PA-31-350,2,8,162.0,Reciprocating


### `PySpark`

## Zapytanie nr 7

### `SQL`

In [122]:
pd.read_sql_query("SELECT tailnum, seats, year FROM planes WHERE seats BETWEEN 100 AND 200 AND year >= 2010", nycflights13_db)

,tailnum,seats,year
0,N127UW,182,2010.0
1,N128UW,182,2010.0
2,N150UW,199,2013.0
3,N151UW,199,2013.0
4,N152UW,199,2013.0
5,N153UW,199,2013.0
6,N154UW,199,2013.0
7,N155UW,199,2013.0
8,N156UW,199,2013.0
9,N157UW,199,2013.0


### `Pandas`

In [180]:
planes.loc[(planes.seats >= 100) & (planes.seats <= 200) & (planes.year >= 2010), :] \
    .reset_index().loc[:, ["tailnum", "seats", "year"]]

,tailnum,seats,year
0,N127UW,182,2010.0
1,N128UW,182,2010.0
2,N150UW,199,2013.0
3,N151UW,199,2013.0
4,N152UW,199,2013.0
5,N153UW,199,2013.0
6,N154UW,199,2013.0
7,N155UW,199,2013.0
8,N156UW,199,2013.0
9,N157UW,199,2013.0


### `PySpark`

## Zapytanie nr 8

### `SQL`

In [127]:
pd.read_sql_query("SELECT * FROM planes WHERE manufacturer IN (\"BOEING\", \"AIRBUS\", \"EMBRAER\") AND seats>300", nycflights13_db)

,index,tailnum,year,type,manufacturer,model,engines,seats,speed,engine
0,49,N1200K,1998.0,Fixed wing multi engine,BOEING,767-332,2,330,None,Turbo-fan
1,50,N1201P,1998.0,Fixed wing multi engine,BOEING,767-332,2,330,None,Turbo-fan
2,69,N121DE,1987.0,Fixed wing multi engine,BOEING,767-332,2,330,None,Turbo-fan
3,117,N136DL,1991.0,Fixed wing multi engine,BOEING,767-332,2,330,None,Turbo-fan
4,122,N137DL,1991.0,Fixed wing multi engine,BOEING,767-332,2,330,None,Turbo-fan
5,183,N143DA,1998.0,Fixed wing multi engine,BOEING,767-332,2,330,None,Turbo-fan
6,241,N1602,1999.0,Fixed wing multi engine,BOEING,767-332,2,330,None,Turbo-fan
7,242,N1603,1999.0,Fixed wing multi engine,BOEING,767-332,2,330,None,Turbo-fan
8,243,N1604R,1999.0,Fixed wing multi engine,BOEING,767-332,2,330,None,Turbo-fan
9,244,N1605,1999.0,Fixed wing multi engine,BOEING,767-332,2,330,None,Turbo-fan


### `Pandas`

In [177]:
planes.loc[(planes.manufacturer.isin(["BOEING", "AIRBUS", "EMBRAER"])) & (planes.seats > 300), :] \
    .reset_index()

,index,tailnum,year,type,manufacturer,model,engines,seats,speed,engine
0,49,N1200K,1998.0,Fixed wing multi engine,BOEING,767-332,2,330,NaN,Turbo-fan
1,50,N1201P,1998.0,Fixed wing multi engine,BOEING,767-332,2,330,NaN,Turbo-fan
2,69,N121DE,1987.0,Fixed wing multi engine,BOEING,767-332,2,330,NaN,Turbo-fan
3,117,N136DL,1991.0,Fixed wing multi engine,BOEING,767-332,2,330,NaN,Turbo-fan
4,122,N137DL,1991.0,Fixed wing multi engine,BOEING,767-332,2,330,NaN,Turbo-fan
5,183,N143DA,1998.0,Fixed wing multi engine,BOEING,767-332,2,330,NaN,Turbo-fan
6,241,N1602,1999.0,Fixed wing multi engine,BOEING,767-332,2,330,NaN,Turbo-fan
7,242,N1603,1999.0,Fixed wing multi engine,BOEING,767-332,2,330,NaN,Turbo-fan
8,243,N1604R,1999.0,Fixed wing multi engine,BOEING,767-332,2,330,NaN,Turbo-fan
9,244,N1605,1999.0,Fixed wing multi engine,BOEING,767-332,2,330,NaN,Turbo-fan


### `PySpark`

## Zapytanie nr 9

### `SQL`

In [131]:
pd.read_sql_query("SELECT manufacturer, COUNT(*) FROM planes WHERE seats > 200 GROUP BY manufacturer", nycflights13_db)

,manufacturer,COUNT(*)
0,AIRBUS,66
1,AIRBUS INDUSTRIE,4
2,BOEING,225


### `Pandas`

In [144]:
planes.loc[(planes.seats > 200), :] \
    .manufacturer.value_counts() \
    .reset_index() \
    .rename(columns={"manufacturer": "count", "index": "manufacturer"})

,manufacturer,count
0,BOEING,225
1,AIRBUS,66
2,AIRBUS INDUSTRIE,4


### `PySpark`

## Zapytanie nr 10

In [140]:
pd.read_sql_query("SELECT manufacturer, COUNT(*) FROM planes GROUP BY manufacturer HAVING COUNT(*) > 10", nycflights13_db)

,manufacturer,COUNT(*)
0,AIRBUS,336
1,AIRBUS INDUSTRIE,400
2,BOEING,1630
3,BOMBARDIER INC,368
4,EMBRAER,299
5,MCDONNELL DOUGLAS,120
6,MCDONNELL DOUGLAS AIRCRAFT CO,103
7,MCDONNELL DOUGLAS CORPORATION,14


### `Pandas`

In [159]:
aa = planes.manufacturer.value_counts()
aa[aa > 10].reset_index()

,index,manufacturer
0,BOEING,1630
1,AIRBUS INDUSTRIE,400
2,BOMBARDIER INC,368
3,AIRBUS,336
4,EMBRAER,299
5,MCDONNELL DOUGLAS,120
6,MCDONNELL DOUGLAS AIRCRAFT CO,103
7,MCDONNELL DOUGLAS CORPORATION,14


### `PySpark`

## Zapytanie nr 11

### `SQL`

In [161]:
pd.read_sql_query("SELECT manufacturer, COUNT(*) FROM planes WHERE seats > 200 GROUP BY manufacturer HAVING COUNT(*) > 10", nycflights13_db)

,manufacturer,COUNT(*)
0,AIRBUS,66
1,BOEING,225


### `Pandas`

In [164]:
d = planes.loc[planes.seats > 200, :].manufacturer.value_counts()
d[d > 10].reset_index()

,index,manufacturer
0,BOEING,225
1,AIRBUS,66


### `PySpark`

## Zapytanie nr 12

### `SQL`

In [168]:
pd.read_sql_query("SELECT manufacturer, COUNT(*) AS howmany FROM planes GROUP BY manufacturer ORDER BY howmany DESC LIMIT 10", nycflights13_db)

,manufacturer,howmany
0,BOEING,1630
1,AIRBUS INDUSTRIE,400
2,BOMBARDIER INC,368
3,AIRBUS,336
4,EMBRAER,299
5,MCDONNELL DOUGLAS,120
6,MCDONNELL DOUGLAS AIRCRAFT CO,103
7,MCDONNELL DOUGLAS CORPORATION,14
8,CANADAIR,9
9,CESSNA,9


### `Pandas`

In [170]:
planes.manufacturer.value_counts() \
    .sort_values(ascending=False) \
    .head(10) \
    .reset_index()

,index,manufacturer
0,BOEING,1630
1,AIRBUS INDUSTRIE,400
2,BOMBARDIER INC,368
3,AIRBUS,336
4,EMBRAER,299
5,MCDONNELL DOUGLAS,120
6,MCDONNELL DOUGLAS AIRCRAFT CO,103
7,MCDONNELL DOUGLAS CORPORATION,14
8,CESSNA,9
9,CANADAIR,9


### `PySpark`

## Zapytanie nr 13

### Pierwszy przykład

#### `SQL`

In [173]:
pd.read_sql_query("SELECT * FROM planes WHERE year >= 2012 ORDER BY year, seats", nycflights13_db)

,index,tailnum,year,type,manufacturer,model,engines,seats,speed,engine
0,1564,N537JB,2012.0,Rotorcraft,ROBINSON HELICOPTER CO,R66,1,5,None,Turbo-shaft
1,815,N348JB,2012.0,Fixed wing multi engine,EMBRAER,ERJ 190-100 IGW,2,20,None,Turbo-fan
2,825,N351JB,2012.0,Fixed wing multi engine,EMBRAER,ERJ 190-100 IGW,2,20,None,Turbo-fan
3,837,N353JB,2012.0,Fixed wing multi engine,EMBRAER,ERJ 190-100 IGW,2,20,None,Turbo-fan
4,2630,N8301J,2012.0,Fixed wing multi engine,BOEING,737-8H4,2,140,None,Turbo-fan
5,2631,N8302F,2012.0,Fixed wing multi engine,BOEING,737-8H4,2,140,None,Turbo-fan
6,2633,N8305E,2012.0,Fixed wing multi engine,BOEING,737-8H4,2,140,None,Turbo-fan
7,2634,N8306H,2012.0,Fixed wing multi engine,BOEING,737-8H4,2,140,None,Turbo-fan
8,2635,N8307K,2012.0,Fixed wing multi engine,BOEING,737-8H4,2,140,None,Turbo-fan
9,2636,N8308K,2012.0,Fixed wing multi engine,BOEING,737-8H4,2,140,None,Turbo-fan


#### `Pandas`

In [175]:
planes.loc[planes.year >= 2012, :].sort_values(["year", "seats"]).reset_index()

,index,tailnum,year,type,manufacturer,model,engines,seats,speed,engine
0,1564,N537JB,2012.0,Rotorcraft,ROBINSON HELICOPTER CO,R66,1,5,NaN,Turbo-shaft
1,815,N348JB,2012.0,Fixed wing multi engine,EMBRAER,ERJ 190-100 IGW,2,20,NaN,Turbo-fan
2,825,N351JB,2012.0,Fixed wing multi engine,EMBRAER,ERJ 190-100 IGW,2,20,NaN,Turbo-fan
3,837,N353JB,2012.0,Fixed wing multi engine,EMBRAER,ERJ 190-100 IGW,2,20,NaN,Turbo-fan
4,2630,N8301J,2012.0,Fixed wing multi engine,BOEING,737-8H4,2,140,NaN,Turbo-fan
5,2631,N8302F,2012.0,Fixed wing multi engine,BOEING,737-8H4,2,140,NaN,Turbo-fan
6,2633,N8305E,2012.0,Fixed wing multi engine,BOEING,737-8H4,2,140,NaN,Turbo-fan
7,2634,N8306H,2012.0,Fixed wing multi engine,BOEING,737-8H4,2,140,NaN,Turbo-fan
8,2635,N8307K,2012.0,Fixed wing multi engine,BOEING,737-8H4,2,140,NaN,Turbo-fan
9,2636,N8308K,2012.0,Fixed wing multi engine,BOEING,737-8H4,2,140,NaN,Turbo-fan


#### PySpark

### Drugi przykład

#### `SQL`

In [183]:
pd.read_sql_query("SELECT * FROM planes WHERE year >= 2012 ORDER BY seats, year", nycflights13_db)

,index,tailnum,year,type,manufacturer,model,engines,seats,speed,engine
0,1564,N537JB,2012.0,Rotorcraft,ROBINSON HELICOPTER CO,R66,1,5,None,Turbo-shaft
1,815,N348JB,2012.0,Fixed wing multi engine,EMBRAER,ERJ 190-100 IGW,2,20,None,Turbo-fan
2,825,N351JB,2012.0,Fixed wing multi engine,EMBRAER,ERJ 190-100 IGW,2,20,None,Turbo-fan
3,837,N353JB,2012.0,Fixed wing multi engine,EMBRAER,ERJ 190-100 IGW,2,20,None,Turbo-fan
4,843,N354JB,2013.0,Fixed wing multi engine,EMBRAER,ERJ 190-100 IGW,2,20,None,Turbo-fan
5,848,N355JB,2013.0,Fixed wing multi engine,EMBRAER,ERJ 190-100 IGW,2,20,None,Turbo-fan
6,859,N358JB,2013.0,Fixed wing multi engine,EMBRAER,ERJ 190-100 IGW,2,20,None,Turbo-fan
7,908,N368JB,2013.0,Fixed wing multi engine,EMBRAER,ERJ 190-100 IGW,2,20,None,Turbo-fan
8,956,N373JB,2013.0,Fixed wing multi engine,EMBRAER,ERJ 190-100 IGW,2,20,None,Turbo-fan
9,989,N374JB,2013.0,Fixed wing multi engine,EMBRAER,ERJ 190-100 IGW,2,20,None,Turbo-fan


#### `Pandas`

In [185]:
planes.loc[planes.year >= 2012, :].sort_values(["seats", "year"]).reset_index()

,index,tailnum,year,type,manufacturer,model,engines,seats,speed,engine
0,1564,N537JB,2012.0,Rotorcraft,ROBINSON HELICOPTER CO,R66,1,5,NaN,Turbo-shaft
1,815,N348JB,2012.0,Fixed wing multi engine,EMBRAER,ERJ 190-100 IGW,2,20,NaN,Turbo-fan
2,825,N351JB,2012.0,Fixed wing multi engine,EMBRAER,ERJ 190-100 IGW,2,20,NaN,Turbo-fan
3,837,N353JB,2012.0,Fixed wing multi engine,EMBRAER,ERJ 190-100 IGW,2,20,NaN,Turbo-fan
4,843,N354JB,2013.0,Fixed wing multi engine,EMBRAER,ERJ 190-100 IGW,2,20,NaN,Turbo-fan
5,848,N355JB,2013.0,Fixed wing multi engine,EMBRAER,ERJ 190-100 IGW,2,20,NaN,Turbo-fan
6,859,N358JB,2013.0,Fixed wing multi engine,EMBRAER,ERJ 190-100 IGW,2,20,NaN,Turbo-fan
7,908,N368JB,2013.0,Fixed wing multi engine,EMBRAER,ERJ 190-100 IGW,2,20,NaN,Turbo-fan
8,956,N373JB,2013.0,Fixed wing multi engine,EMBRAER,ERJ 190-100 IGW,2,20,NaN,Turbo-fan
9,989,N374JB,2013.0,Fixed wing multi engine,EMBRAER,ERJ 190-100 IGW,2,20,NaN,Turbo-fan


## Zapytanie nr 14

### `SQL`

In [13]:
pd.read_sql_query("""SELECT planes.*, airlines.* FROM  
   (SELECT DISTINCT carrier, tailnum FROM flights) AS cartail 
JOIN planes ON cartail.tailnum=planes.tailnum 
JOIN airlines ON cartail.carrier=airlines.carrier""", nycflights13_db)

,index,tailnum,year,type,manufacturer,model,engines,seats,speed,engine,index,carrier,name
0,0,N10156,2004.0,Fixed wing multi engine,EMBRAER,EMB-145XR,2,55,NaN,Turbo-fan,5,EV,ExpressJet Airlines Inc.
1,1,N102UW,1998.0,Fixed wing multi engine,AIRBUS INDUSTRIE,A320-214,2,182,NaN,Turbo-fan,12,US,US Airways Inc.
2,2,N103US,1999.0,Fixed wing multi engine,AIRBUS INDUSTRIE,A320-214,2,182,NaN,Turbo-fan,12,US,US Airways Inc.
3,3,N104UW,1999.0,Fixed wing multi engine,AIRBUS INDUSTRIE,A320-214,2,182,NaN,Turbo-fan,12,US,US Airways Inc.
4,4,N10575,2002.0,Fixed wing multi engine,EMBRAER,EMB-145LR,2,55,NaN,Turbo-fan,5,EV,ExpressJet Airlines Inc.
5,5,N105UW,1999.0,Fixed wing multi engine,AIRBUS INDUSTRIE,A320-214,2,182,NaN,Turbo-fan,12,US,US Airways Inc.
6,6,N107US,1999.0,Fixed wing multi engine,AIRBUS INDUSTRIE,A320-214,2,182,NaN,Turbo-fan,12,US,US Airways Inc.
7,7,N108UW,1999.0,Fixed wing multi engine,AIRBUS INDUSTRIE,A320-214,2,182,NaN,Turbo-fan,12,US,US Airways Inc.
8,8,N109UW,1999.0,Fixed wing multi engine,AIRBUS INDUSTRIE,A320-214,2,182,NaN,Turbo-fan,12,US,US Airways Inc.
9,9,N110UW,1999.0,Fixed wing multi engine,AIRBUS INDUSTRIE,A320-214,2,182,NaN,Turbo-fan,12,US,US Airways Inc.


### `Pandas`

In [14]:
cartail = flights.loc[:, ["carrier", "tailnum"]].drop_duplicates().reset_index().loc[:, ["carrier", "tailnum"]]
A = pd.merge(cartail, planes, on="tailnum").reset_index()
pd.merge(A, airlines, on="carrier").loc[:, list(planes.columns) + list(airlines.columns)].reset_index()

,index,tailnum,year,type,manufacturer,model,engines,seats,speed,engine,carrier,name
0,0,N14228,1999.0,Fixed wing multi engine,BOEING,737-824,2,149,NaN,Turbo-fan,UA,United Air Lines Inc.
1,1,N24211,1998.0,Fixed wing multi engine,BOEING,737-824,2,149,NaN,Turbo-fan,UA,United Air Lines Inc.
2,2,N39463,2012.0,Fixed wing multi engine,BOEING,737-924ER,2,191,NaN,Turbo-fan,UA,United Air Lines Inc.
3,3,N29129,1998.0,Fixed wing multi engine,BOEING,757-224,2,178,NaN,Turbo-jet,UA,United Air Lines Inc.
4,4,N53441,NaN,Fixed wing multi engine,BOEING,737-924ER,2,191,NaN,Turbo-fan,UA,United Air Lines Inc.
5,5,N76515,2008.0,Fixed wing multi engine,BOEING,737-824,2,149,NaN,Turbo-fan,UA,United Air Lines Inc.
6,6,N53442,2009.0,Fixed wing multi engine,BOEING,737-924ER,2,191,NaN,Turbo-fan,UA,United Air Lines Inc.
7,7,N459UA,2000.0,Fixed wing multi engine,AIRBUS INDUSTRIE,A320-232,2,200,NaN,Turbo-fan,UA,United Air Lines Inc.
8,8,N33289,2004.0,Fixed wing multi engine,BOEING,737-824,2,149,NaN,Turbo-fan,UA,United Air Lines Inc.
9,9,N497UA,2002.0,Fixed wing multi engine,AIRBUS,A320-232,2,200,NaN,Turbo-fan,UA,United Air Lines Inc.


Powyższe dwie ramki różnią się jedną kolumną, gdyż w pierwszej (z SQL) ramce danych jest brany index z dwóch złączeń.

## Zapytanie nr 15

### `SQL`

In [37]:
pd.read_sql_query("""SELECT flights2.*, weather2.atemp, weather2.ahumid, weather2.apressure FROM 
   (SELECT * FROM flights WHERE origin='EWR') AS flights2 
LEFT JOIN 
   (SELECT year, month, day, AVG(temp) AS atemp, 
     AVG(humid) AS ahumid, AVG(pressure) AS apressure 
    FROM weather WHERE origin='EWR' GROUP BY year, month, day) AS weather2 
ON flights2.year=weather2.year 
  AND flights2.month=weather2.month 
  AND flights2.day=weather2.day""", nycflights13_db)

,index,year,month,day,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,...,origin,dest,air_time,distance,hour,minute,time_hour,weather2.atemp,weather2.ahumid,weather2.apressure
0,0,2013,1,1,517.0,515,2.0,830.0,819,11.0,...,EWR,IAH,227.0,1400,5,15,2013-01-01 05:00:00,38.4800,58.386087,1012.442857
1,5,2013,1,1,554.0,558,-4.0,740.0,728,12.0,...,EWR,ORD,150.0,719,5,58,2013-01-01 05:00:00,38.4800,58.386087,1012.442857
2,6,2013,1,1,555.0,600,-5.0,913.0,854,19.0,...,EWR,FLL,158.0,1065,6,0,2013-01-01 06:00:00,38.4800,58.386087,1012.442857
3,13,2013,1,1,558.0,600,-2.0,923.0,937,-14.0,...,EWR,SFO,361.0,2565,6,0,2013-01-01 06:00:00,38.4800,58.386087,1012.442857
4,16,2013,1,1,559.0,600,-1.0,854.0,902,-8.0,...,EWR,LAS,337.0,2227,6,0,2013-01-01 06:00:00,38.4800,58.386087,1012.442857
5,19,2013,1,1,601.0,600,1.0,844.0,850,-6.0,...,EWR,PBI,147.0,1023,6,0,2013-01-01 06:00:00,38.4800,58.386087,1012.442857
6,22,2013,1,1,606.0,610,-4.0,858.0,910,-12.0,...,EWR,MIA,152.0,1085,6,10,2013-01-01 06:00:00,38.4800,58.386087,1012.442857
7,24,2013,1,1,607.0,607,0.0,858.0,915,-17.0,...,EWR,MIA,157.0,1085,6,7,2013-01-01 06:00:00,38.4800,58.386087,1012.442857
8,25,2013,1,1,608.0,600,8.0,807.0,735,32.0,...,EWR,ORD,139.0,719,6,0,2013-01-01 06:00:00,38.4800,58.386087,1012.442857
9,29,2013,1,1,615.0,615,0.0,833.0,842,-9.0,...,EWR,ATL,120.0,746,6,15,2013-01-01 06:00:00,38.4800,58.386087,1012.442857


### `Pandas`

In [63]:
flights2 = flights.loc[flights.origin == 'EWR', :]
weather2 = pd.DataFrame([(key[0], key[1], key[2], group.temp.mean(), group.humid.mean(), group.pressure.mean()) \
                         for key, group in weather.loc[weather.origin == 'EWR', :].groupby(["year", "month", "day"])],
                       columns=["year", "month", "day", "atemp", "ahumid", "apressure"])
pd.merge(flights2, weather2, on=["year", "month", "day"], how="left") \
    .loc[:, list(flights2.columns) + ["atemp", "ahumid", "apressure"]] \
    .reset_index()

,index,year,month,day,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,...,origin,dest,air_time,distance,hour,minute,time_hour,atemp,ahumid,apressure
0,0,2013,1,1,517.0,515,2.0,830.0,819,11.0,...,EWR,IAH,227.0,1400,5,15,2013-01-01 05:00:00,38.4800,58.386087,1012.442857
1,1,2013,1,1,554.0,558,-4.0,740.0,728,12.0,...,EWR,ORD,150.0,719,5,58,2013-01-01 05:00:00,38.4800,58.386087,1012.442857
2,2,2013,1,1,555.0,600,-5.0,913.0,854,19.0,...,EWR,FLL,158.0,1065,6,0,2013-01-01 06:00:00,38.4800,58.386087,1012.442857
3,3,2013,1,1,558.0,600,-2.0,923.0,937,-14.0,...,EWR,SFO,361.0,2565,6,0,2013-01-01 06:00:00,38.4800,58.386087,1012.442857
4,4,2013,1,1,559.0,600,-1.0,854.0,902,-8.0,...,EWR,LAS,337.0,2227,6,0,2013-01-01 06:00:00,38.4800,58.386087,1012.442857
5,5,2013,1,1,601.0,600,1.0,844.0,850,-6.0,...,EWR,PBI,147.0,1023,6,0,2013-01-01 06:00:00,38.4800,58.386087,1012.442857
6,6,2013,1,1,606.0,610,-4.0,858.0,910,-12.0,...,EWR,MIA,152.0,1085,6,10,2013-01-01 06:00:00,38.4800,58.386087,1012.442857
7,7,2013,1,1,607.0,607,0.0,858.0,915,-17.0,...,EWR,MIA,157.0,1085,6,7,2013-01-01 06:00:00,38.4800,58.386087,1012.442857
8,8,2013,1,1,608.0,600,8.0,807.0,735,32.0,...,EWR,ORD,139.0,719,6,0,2013-01-01 06:00:00,38.4800,58.386087,1012.442857
9,9,2013,1,1,615.0,615,0.0,833.0,842,-9.0,...,EWR,ATL,120.0,746,6,15,2013-01-01 06:00:00,38.4800,58.386087,1012.442857
